```bash
DATA CLEANING
```

In [13]:
import yfinance as yf
import pandas as pd

sp500_raw = yf.download('^GSPC', start='1985-01-01', end='2024-12-31', interval='1mo')
nasdaq_raw = yf.download('^IXIC', start='1985-01-01', end='2024-12-31', interval='1mo')
dow_raw = yf.download('^DJI', start='1985-01-01', end='2024-12-31', interval='1mo')

sp500 = sp500_raw[['Close']].rename(columns={'Close': 'S&P 500'})
nasdaq = nasdaq_raw[['Close']].rename(columns={'Close': 'NASDAQ'})
dow = dow_raw[['Close']].rename(columns={'Close': 'Dow Jones'})
market_data = sp500.join([nasdaq, dow]).reset_index()

market_data_quarterly = market_data[market_data['Date'].dt.month.isin([1, 4, 7, 10])].reset_index(drop=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [14]:
cpi = pd.read_csv("CPI data.csv") #quarterly, (jan 1 1960 - November 1 2023)
ir = pd.read_csv("Real interest rate.csv") #monthly, (Jan 1 1982 - March 1 2025)
unemployment = pd.read_csv("Unemployment data.csv") #monthly, (Jan 1 1984 - Febrary 1 2025)
gdp_growth = pd.read_csv("GDP growth rate.csv") #quarterly, April 1947 - Nov 2024
m2_supply = pd.read_csv("M2 money supply.csv") #weekly, 1980 11-03 to 2025 03-03
fedfunds = pd.read_csv("FEDFUNDS.csv") #monthly July 1954 to March 2025

In [15]:
cpi = cpi.rename(columns={'observation_date': 'Date', 'CPALTT01USQ661N': 'CPI'})
ir = ir.rename(columns={'observation_date': 'Date', 'REAINTRATREARAT10Y': 'RealInterestRate'})
unemployment = unemployment.rename(columns={'observation_date': 'Date', 'UNRATE': 'UnemploymentRate'})

cpi['Date'] = pd.to_datetime(cpi['Date'])
ir['Date'] = pd.to_datetime(ir['Date'])
unemployment['Date'] = pd.to_datetime(unemployment['Date'])

ir_q = ir.set_index('Date').resample('QS').first().reset_index()
unemp_q = unemployment.set_index('Date').resample('QS').first().reset_index()

cpi = cpi.reset_index(drop=True)
ir_q = ir_q.reset_index(drop=True)
unemp_q = unemp_q.reset_index(drop=True)
market_data_quarterly = market_data_quarterly.reset_index(drop=True)

market_data_quarterly.columns = [' '.join(col).strip() if isinstance(col, tuple) else col for col in market_data_quarterly.columns]

market_data_quarterly['Date'] = pd.to_datetime(market_data_quarterly['Date'])

# Merge all datasets on Date
merged_data = market_data_quarterly.merge(cpi, on='Date', how='inner')
merged_data = merged_data.merge(ir_q, on='Date', how='inner')
merged_data = merged_data.merge(unemp_q, on='Date', how='inner')

merged_data = merged_data[merged_data['Date'] >= '1992-01-01'].reset_index(drop=True)

gdp_growth = gdp_growth.rename(columns={'observation_date': 'Date', 'A191RL1Q225SBEA': 'GDP_Growth'})
m2_supply = m2_supply.rename(columns={'observation_date': 'Date', 'WM2NS': 'M2_Supply'})
fedfunds = fedfunds.rename(columns={'observation_date': 'Date', 'FEDFUNDS': 'FedFundsRate'})

gdp_growth['Date'] = pd.to_datetime(gdp_growth['Date'])
m2_supply['Date'] = pd.to_datetime(m2_supply['Date'])
fedfunds['Date'] = pd.to_datetime(fedfunds['Date'])

m2_q = m2_supply.set_index('Date').resample('QS').first().reset_index()
fedfunds_q = fedfunds.set_index('Date').resample('QS').first().reset_index()

gdp_growth = gdp_growth[gdp_growth['Date'] >= '1992-01-01'].reset_index(drop=True)
m2_q = m2_q[m2_q['Date'] >= '1992-01-01'].reset_index(drop=True)
fedfunds_q = fedfunds_q[fedfunds_q['Date'] >= '1992-01-01'].reset_index(drop=True)

merged_data = merged_data.merge(gdp_growth, on='Date', how='inner')
merged_data = merged_data.merge(m2_q, on='Date', how='inner')
merged_data = merged_data.merge(fedfunds_q, on='Date', how='inner')

merged_data

,Date,S&P 500 ^GSPC,NASDAQ ^IXIC,Dow Jones ^DJI,CPALWE01USQ661N,RealInterestRate,UnemploymentRate,GDP_Growth,M2_Supply,FedFundsRate
0,1992-01-01,408.779999,620.210022,3223.399902,58.870052,3.198542,7.3,4.9,3398.6,4.03
1,1992-04-01,414.950012,578.679993,3359.100098,59.387717,3.789860,7.4,4.4,3434.1,3.73
2,1992-07-01,424.209991,580.830017,3393.800049,59.862244,3.353099,7.7,4.0,3407.1,3.25
3,1992-10-01,418.679993,605.169983,3226.300049,60.279252,2.728044,7.3,4.2,3413.8,3.10
4,1993-01-01,438.779999,696.340027,3310.000000,60.696260,3.267908,7.3,0.7,3455.2,3.02
...,...,...,...,...,...,...,...,...,...,...
123,2022-10-01,3871.979980,10988.150391,32732.949219,126.044174,1.797332,3.6,3.4,21429.5,3.08
124,2023-01-01,4076.600098,11584.549805,34086.039062,127.208058,1.783099,3.5,2.8,21364.4,4.33
125,2023-04-01,4169.479980,12226.580078,34098.160156,128.770113,1.443914,3.4,2.4,20924.5,4.83
126,2023-07-01,4588.959961,14346.019531,35559.531250,129.949384,1.426047,3.5,4.4,20744.8,5.12
